In [6]:
import pandas as pd
import numpy as np

data_path = "M:/Dissertation/Data/"

##### **Reading the Data and Selecting Columns**

In [7]:
data = pd.read_csv(data_path+"Forex_Data(Raw).csv")
data = data.drop(["XAU/GBP","XAU/EUR"],axis=1)
data = data.rename(columns={"XAUUSD":"XAU/USD"})

##### **Resampling the data from Minute to Hour**

In [8]:
# Breaking down the Date column in components
data["Date"] = pd.to_datetime(data["Date"],format="%Y-%m-%d %H:%M:%S")
data["Year"] = data["Date"].dt.year
data["Month"] = data["Date"].dt.month
data["Day"] = data["Date"].dt.day
data["Hour"] = data["Date"].dt.hour

# Groupby w.r.t Hour
data = data.groupby(by=["Year","Month","Day","Hour"],as_index=False)[['EUR/USD','EUR/GBP','GBP/USD','XAU/USD']].mean()

# Reconstruct the Date column
data["Date"] = pd.to_datetime(data["Year"].astype(str)+"-"+
                              data["Month"].astype(str)+"-"+
                              data["Day"].astype(str)+" "+
                              data["Hour"].astype(str)+":00:00",format="%Y-%m-%d %H:%M:%S")

# Save Hourly Data
data[['Date','EUR/USD','EUR/GBP','GBP/USD','XAU/USD']].to_csv(data_path+"Forex_Data(Hourly).csv",index=False)

##### **Removing the Large shifts in Prices**

In [9]:
# Iterating over each currency
for col in ['EUR/USD','EUR/GBP','GBP/USD','XAU/USD']:
    print(f"The Mean Hourly Difference before adjustment in {col} is {data[col].diff(1).mean():.2e}")
    counter = 0
    
    # Setting thresholds using mean of history
    for i in range(len(data)-1,2,-1):
        prev = data[col].iloc[:i]
        thr = 0.0025*np.mean(prev)
        dif = data[col].iloc[i]-data[col].iloc[i-1]
        if (dif >= thr) or (dif <= -thr):
            counter += 1
            if dif > 0:
                data.loc[:i-1, col] += dif - (thr)
            else:
                data.loc[:i-1, col] += dif + (thr)

    # Calculating Price Returns and Lags
    data[col+"_R"] = (data[col].shift(-1)/data[col])-1
    data[col+"_T"] = data[col].shift(-1)
    
    # Printing the Statistics before and after adjustment
    print(f"Percentage of Adjusted Data Points in {col} is {(counter/len(data))*100:.2f} %")
    print(f"The Mean Hourly Difference after adjustment in {col} is {data[col].diff(1).mean():.2e} \n")

# Saving the Data
data[['Date','EUR/USD','EUR/GBP','GBP/USD','XAU/USD',
      'EUR/USD_T','EUR/GBP_T','GBP/USD_T','XAU/USD_T',
      'EUR/USD_R','EUR/GBP_R','GBP/USD_R','XAU/USD_R']].to_csv(data_path+"Forex_Data.csv",index=False)

The Mean Hourly Difference before adjustment in EUR/USD is -4.81e-06
Percentage of Adjusted Data Points in EUR/USD is 1.43 %
The Mean Hourly Difference after adjustment in EUR/USD is -5.04e-06 

The Mean Hourly Difference before adjustment in EUR/GBP is 8.74e-07
Percentage of Adjusted Data Points in EUR/GBP is 1.94 %
The Mean Hourly Difference after adjustment in EUR/GBP is 9.69e-08 

The Mean Hourly Difference before adjustment in GBP/USD is -7.35e-06
Percentage of Adjusted Data Points in GBP/USD is 1.75 %
The Mean Hourly Difference after adjustment in GBP/USD is -6.33e-06 

The Mean Hourly Difference before adjustment in XAU/USD is -4.13e-03
Percentage of Adjusted Data Points in XAU/USD is 5.48 %
The Mean Hourly Difference after adjustment in XAU/USD is -7.08e-03 

